In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectConfig
from loss_funcs import *
from algs.random_sampling import RandomSampling
from algs.particle_swarm import ParticleSwarm
from algs.simulated_annealing import SimulatedAnnealing

# Create a camera configuration
cam_config = CameraConfig(position=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

object_init_position = (0, 1.3, 0.3)

search_alg = ParticleSwarm(sim_viewer, MSE())

random_orientations = np.random.uniform(0, 2 * np.pi, size=(50, 3))

for orient in random_orientations:

    img_world, obj_config = world_viewer.get_view(ObjectConfig(orient, object_init_position), depth=True)
    print(img_world.shape)
    print(img_world.dtype)
    print(img_world.min(), img_world.max())

    img_world, obj_config = world_viewer.get_view_cropped(ObjectConfig(orient, object_init_position))

    best_orient = search_alg.find_orientation(
        ref_img=img_world, ref_position=obj_config.position, config=ParticleSwarm.Config(time_limit=100)
    )

    best_image, _ = sim_viewer.get_view_cropped(ObjectConfig(best_orient, obj_config.position))

    cv.imshow("real_img", img_world)
    cv.imshow("sim_img", best_image)
    cv.waitKey(0)

In [ ]:
cv.destroyAllWindows()